In [15]:
import yfinance as yf
import pandas as pd

In [3]:
# 讀取景氣資料
df = pd.read_csv('777.csv')

# 2️⃣ 確保 `Date` 是 datetime 格式
df["Date"] = pd.to_datetime(df["Date"], format="%Y/%m/%d")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    241 non-null    datetime64[ns]
 1   lable   241 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 3.9 KB


In [13]:
# 建立每日 DataFrame（展開所有 2 月的日期）
start_date = df["Date"].min()  # 最早的日期
end_date = df["Date"].max()  # 最後的日期
daily_df = pd.DataFrame({"Date": pd.date_range(start=start_date, end=end_date, freq='D')})

# 建立「對應的每月 1 號」標籤
daily_df["對應每月1日"] = pd.to_datetime(daily_df["Date"].dt.to_period("M").astype(str) + "-01")

# 使用 `merge()` 將 `label` 值對應到每天
daily_df = daily_df.merge(df, left_on="對應每月1日", right_on="Date", suffixes=("", "_每月1日")).drop(columns=["Date_每月1日"])

daily_df

,Date,對應每月1日,lable
0,2004-12-01,2004-12-01,2
1,2004-12-02,2004-12-01,2
2,2004-12-03,2004-12-01,2
3,2004-12-04,2004-12-01,2
4,2004-12-05,2004-12-01,2
...,...,...,...
7301,2024-11-27,2024-11-01,3
7302,2024-11-28,2024-11-01,3
7303,2024-11-29,2024-11-01,3
7304,2024-11-30,2024-11-01,3


In [56]:
# 匯入股票資料
df_stock = pd.read_csv('SPYdata.csv')
df_stock["Trade_date"] = pd.to_datetime(df_stock["Trade_date"])
# 使用 INNER JOIN（只保留兩個 DataFrame 都有的日期）
df_merged = df_stock.merge(daily_df, left_on="Trade_date", right_on="Date", how="inner")
df_merged = df_merged.drop(['Date','對應每月1日'],axis=1)
df_merged

/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_97722/2151163312.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_stock["Trade_date"] = pd.to_datetime(df_stock["Trade_date"])


,Trade_date,SPY,lable
0,2004-12-01,85.9551,2
1,2004-12-02,86.0273,2
2,2004-12-03,85.9696,2
3,2004-12-06,85.9407,2
4,2004-12-07,85.1405,2
...,...,...,...
4112,2021-04-05,406.3600,3
4113,2021-04-06,406.1200,3
4114,2021-04-07,406.5900,3
4115,2021-04-08,408.5200,3


In [ ]:
# 儲存為 CSV 檔案
df_merged.to_csv("merged_data.csv", index=False, encoding="utf-8-sig")